# S2W5D4: 组装时刻 - 骨架与肌肉

今日任务：

1. **残差连接 (Residual Connection)**
2. **层归一化 (Layer Normalization)**
3. **前馈网络 (Feed-Forward Network)**

## 1 理论三剑客

### 1.1 残差连接（Add）

- **公式**：$Output = x + \text{Sublayer}(x)$
- **作用**：
    - 深度神经网络最怕梯度传不回去，有了加号，梯度可以直接沿着$x$这条“高速公路”无损地传回到底层。
    - 这是ResNet的核心遗产，也是现代深度学习能做深的基础。

### 1.2 层归一化（Layer Normalization - Norm）

- **公式**：$\text{LayerNorm}(x)$
- **作用**：
    - 把每个样本的特征向量拉成均值为 0，方差为 1 的分布。
    - **面试必问**: **为什么不用 BatchNorm (BN)？**
        - **BN (按批归一化)**: 依赖 Batch Size，且假设同一个 Batch 里的数据分布一致。这对 NLP 不友好（句子长度不一，Token 差异大）。
        - **LN (按层归一化)**: “自己管自己”。不管 Batch 有多大，我只把**这一句话**里的特征做归一化。这对 RNN/Transformer 这种变长序列最合适。

### 1.3 前馈网络

- **结构**：`Linear` -\> `ReLU` -\> `Linear`
- **作用**：
    - Attention 负责“看”和“收集信息”（你是谁，我在哪）。
    - FFN 负责“思考”和“处理信息”（我该怎么理解这些信息）。
    - **维度变化**: 通常会先把维度放大 4 倍（$512 \rightarrow 2048$），然后再缩回来（$2048 \rightarrow 512$）。这叫 **“升维打击”**，为了在这个高维空间里理清非线性关系。

## 2 代码实现（Building Blocks）

在这里将实现两个类：`PositionwiseFeedForward` 和 `EncoderLayer`。

### 2.1 前馈网络（FFN）

In [7]:
import torch
import torch.nn as nn

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        d_model：模型输入维度（512）
        d_ff：隐藏层的维度（通常是4倍d_model=2048）
        """
        super(PositionwiseFeedForward,self).__init__()
        # 第一层：线性变换 + ReLU
        self.w_1 = nn.Linear(d_model, d_ff)
        # 第二层：线性变换
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU() # 或者 GELU (BERT常用)

    def forward(self, x):
        # x: [Batch, Seq, d_model]
        
        # 1. 升维: 512 -> 2048
        # 公式: ReLU(xW1 + b1)W2 + b2
        inter = self.activation(self.w_1(x))
        inter = self.dropout(inter)
        
        # 2. 降维: 2048 -> 512
        output = self.w_2(inter)
        
        return output

## 2.2 完整的 Encoder Layer (组装！)

这是 Transformer 中重复堆叠了 $N=6$ 次的那个单元。
结构口诀：**Attention -\> Add & Norm -\> FFN -\> Add & Norm**

In [8]:
import os
import sys
import torch.nn as nn

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.transformer_components import MultiHeadAttention

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()

        # 1. Self-Attention 模块
        self.self_attn = MultiHeadAttention(d_model, n_heads)

        # 2. Feed-Forward 模块
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)

        # 3. 两个Add & Norm模块
        # LayerNorm的参数是特征维度d_model
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        # 4. Dropout (防止过拟合的神器)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        """
        x: [Batch, Seq, d_model]
        mask: [Batch, 1, 1, Seq] (用于遮挡 Pad)
        """
        # --- 子层1：Multi-Head Attention ---
        # 1. 计算Attention
        attn_output, _ = self.self_attn(x, x, x, mask)

        # 2. Add & Norm
        # 残差连接: x + Dropout(Sublayer(x))
        # 这里的顺序是 Post-Norm: Norm(x + Sublayer(x)) -> 原始论文写法
        # 现代很多模型(GPT)用 Pre-Norm: x + Sublayer(Norm(x))
        # 我们按经典的来：
        x = self.norm1(x + self.dropout(attn_output))

        # --- 子层 2: Feed-Forward ---
        # 1. 计算 FFN
        ff_output = self.feed_forward(x)
        
        # 2. Add & Norm
        x = self.norm2(x + self.dropout(ff_output))
        
        return x

## 3 验证与测试 (Verification)

验证它是积木的关键：**输入进去是什么形状，吐出来还得是什么形状**。

In [9]:
d_model = 512
n_heads = 8
d_ff = 2048 # 4倍
dropout = 0.1
batch_size = 2
seq_len = 10

# 1. 实例化 EncoderLayer
encoder_layer = EncoderLayer(d_model, n_heads, d_ff, dropout)

# 2. 创建假数据
x = torch.randn(batch_size, seq_len, d_model)

# 3. 前向传播
output = encoder_layer(x)

# 4. 检查
print("Input shape:", x.shape)
print("Output shape:", output.shape)

# 验证输入输出同构
assert x.shape == output.shape
print("✅ EncoderLayer 测试通过！它可以无限堆叠了。")

# 查看参数量 (感受一下层级)
num_params = sum(p.numel() for p in encoder_layer.parameters())
print(f"这一层 Encoder 包含了 {num_params} 个参数。")

Input shape: torch.Size([2, 10, 512])
Output shape: torch.Size([2, 10, 512])
✅ EncoderLayer 测试通过！它可以无限堆叠了。
这一层 Encoder 包含了 3152384 个参数。


## 4 深度解析 & 面试考点 (Interview Checkpoints)

### Q1: LayerNorm 和 BatchNorm 到底有什么本质区别？(高频)

  * **形象比喻**:
      * **BatchNorm**: 考试结束后，老师算全班同学(Batch)的平均分。你的分数是相对于全班的排名。如果 Batch 很小（只有你一个人），这平均分就没意义了。
      * **LayerNorm**: 考试结束后，算**你自己**(Sample)语数英三科的平均分。不管班里有多少人，你的“偏科程度”是确定的。
  * **结论**: NLP 样本之间独立性强，且长度不一，不适合横向比较（BN），只适合纵向标准化（LN）。

### Q2: 为什么要先把维度放大 4 倍再缩回来 (FFN)？

  * **SVM 思想**: 低维空间线性不可分的数据，映射到高维空间往往就线性可分了。
  * **记忆容量**: FFN 的两层 Linear 占据了 Transformer 模型 **2/3 的参数量**。它是模型存储“知识”的主要场所（比如“巴黎是法国的首都”这种知识往往存在 FFN 的权重里）。

### Q3: 这里的 Add & Norm 顺序问题？(进阶)

  * **Post-Norm (原始论文)**: `Norm(x + SubLayer(x))`。也就是我们今天写的。优点是收敛后效果好，缺点是训练初期梯度容易炸（需要 Warmup）。
  * **Pre-Norm (GPT/Llama)**: `x + SubLayer(Norm(x))`。把 Norm 放在子层前面。优点是训练极其稳定，不需要 Warmup 也能训，是现在大模型的标配。



## 🚀 今日行动

1.  **代码**: 将 `PositionwiseFeedForward` 和 `EncoderLayer` 写入 Notebook 并跑通。
2.  **工程**: 测试通过后，请把这两个类也追加到你的 `src/models/transformer_components.py` 文件中，让你的军火库越来越丰富。
3.  **具身思考**:
      * 在机器人中，`d_model` 代表了它对世界的理解带宽。
      * `EncoderLayer` 堆得越多，它的推理能力越强（能理解越复杂的指令）。
      * 但是，层数越多，推理延迟（Latency）越高。作为组长，你需要在\*\*“变聪明”**和**“反应快”\*\*之间做权衡。

准备好后，请告诉我。明天（Day 5），我们将把 $N$ 个 EncoderLayer 堆起来，加上 Embedding，组成**完整的 Transformer Encoder**，也就是大名鼎鼎的 **BERT** 架构！